<a href="https://colab.research.google.com/github/moodv/data-analytics-portfolio/blob/main/Project2-online-retail/notebooks/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


df = pd.read_excel('OnlineRetail.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [11]:
df.describe(include="all")
nulls= df.isnull().sum()
duplicate = df.duplicated().sum()

print(f"Nulls: {nulls}")
print(f"Duplicates: {duplicate}")


Nulls: InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133582
Country             0
TotalPrice          0
Year                0
Month               0
Day                 0
Weekday             0
Time                0
Hour                0
dtype: int64
Duplicates: 0


In [3]:
# 1. Standardize text columns
df['Description'] = df['Description'].str.lower()
df['Country'] = df['Country'].str.lower()

# 2. Drop nulls in Description
df = df.dropna(subset=['Description'])

# 3. Drop exact duplicate rows
df = df.drop_duplicates()

# 4. Add TotalPrice
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# 5. Ensure InvoiceDate is datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

# 6. Extract date features
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['Weekday'] = df['InvoiceDate'].dt.day_name()
df['Time'] = df['InvoiceDate'].dt.time
df['Hour'] = df['InvoiceDate'].dt.hour




In [12]:
df.info()
df.head()
print(f"Nulls: {nulls}")
print(f"Duplicates: {duplicate}")

<class 'pandas.core.frame.DataFrame'>
Index: 535186 entries, 0 to 541908
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    535186 non-null  object        
 1   StockCode    535186 non-null  object        
 2   Description  535186 non-null  object        
 3   Quantity     535186 non-null  int64         
 4   InvoiceDate  535186 non-null  datetime64[ns]
 5   UnitPrice    535186 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      535186 non-null  object        
 8   TotalPrice   535186 non-null  float64       
 9   Year         535186 non-null  int32         
 10  Month        535186 non-null  int32         
 11  Day          535186 non-null  int32         
 12  Weekday      535186 non-null  object        
 13  Time         535186 non-null  object        
 14  Hour         535186 non-null  int32         
dtypes: datetime64[ns](1), float64(3), int32